In [1]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

### Cargo los datos y sets

In [2]:
training_labels = pd.read_csv("labels_training_set.csv", low_memory=False)
labels_predict = pd.read_csv("trocafone_kaggle_test.csv", low_memory=False)
events_data = pd.read_csv("data_set.csv", low_memory = False)

### Comenzamos con el set de entrenamiento

In [3]:
set_entrenamiento = pd.merge(events_data, training_labels, on ='person', how = 'inner')

In [4]:
set_entrenamiento = set_entrenamiento.drop(columns = ['person'])
set_entrenamiento.head()

,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,...,promedio sku,promedio de TFIdf,Direct,Email,Organic,Paid,Referral,Social,Unknown,label
0,0,0,3,0,1,0,0,0,0,0,...,8.066940,0.000000,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0
1,1,0,1,0,1,0,1,9,0,3,...,7.757806,1.168325,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
2,5,0,1,0,4,0,0,4,0,4,...,7.759906,1.277478,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
3,29,165,15,2,28,0,13,11,0,189,...,7.541525,1.163643,0.0,0.0,2.0,16.0,1.0,0.0,0.0,0
4,0,1,2,1,1,0,0,0,0,2,...,9.134138,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [5]:
X = set_entrenamiento.drop(columns = ['label'])
y = set_entrenamiento["label"]

In [6]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=123)

In [7]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [8]:
xg_reg = xgb.XGBRegressor(objective ='binary:logistic', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 5, n_estimators = 10)

In [9]:
xg_reg.fit(x_train,y_train)

XGBRegressor(alpha=5, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.3, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=10,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [10]:
preds = xg_reg.predict(x_test)
preds = preds.tolist()
df = pd.DataFrame(preds)
df.describe()

,0
count,7766.000000
mean,0.210721
std,0.025448
min,0.189592
25%,0.194026
50%,0.199385
75%,0.221225
max,0.468162


In [11]:
df = df[0].map(lambda x: 1 if x > 0.328877 else 0);
df = df.to_frame()
df.rename(columns = {0: 'label'},inplace = True) 

In [12]:
df['label'].value_counts()

0    7732
1      34
Name: label, dtype: int64

In [13]:
accuracy_score(df,y_test)

0.9455318053051764

### Comenzamos con el set de test

In [14]:
set_test = pd.merge(events_data,labels_predict,on = "person", how = "right")
set_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19415 entries, 0 to 19414
Data columns (total 24 columns):
person               19415 non-null object
ad campaign hit      19415 non-null int64
brand listing        19415 non-null int64
checkout             19415 non-null int64
conversion           19415 non-null int64
generic listing      19415 non-null int64
lead                 19415 non-null int64
search engine hit    19415 non-null int64
searched products    19415 non-null int64
staticpage           19415 non-null int64
viewed product       19415 non-null int64
visited site         19415 non-null int64
promedio dias        19415 non-null float64
condicion            19415 non-null int64
promedio hora        19415 non-null float64
promedio sku         19415 non-null float64
promedio de TFIdf    19415 non-null float64
Direct               19415 non-null float64
Email                19415 non-null float64
Organic              19415 non-null float64
Paid                 19415 non-null 

In [15]:
personas = set_test["person"]
set_test = set_test.drop(columns=['person'])
set_test = set_test.fillna(0)

In [16]:
predicts_kaggel = xg_reg.predict(set_test)
set_test.tail()

,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,staticpage,viewed product,...,promedio hora,promedio sku,promedio de TFIdf,Direct,Email,Organic,Paid,Referral,Social,Unknown
19410,0,0,0,0,0,0,1,0,0,19,...,14.476190,9.386273,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0
19411,1,0,1,0,0,0,0,0,0,1,...,17.000000,8.569121,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0
19412,0,0,0,0,0,0,1,0,0,0,...,19.000000,-inf,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0
19413,7,1,1,1,12,0,12,8,3,42,...,17.896907,7.909698,1.050893,0.0,0.0,7.0,3.0,0.0,0.0,0.0
19414,1,2,1,0,0,0,1,0,0,4,...,22.000000,7.790903,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [17]:
predicts_kaggel = predicts_kaggel.tolist()
df_kaggel = pd.DataFrame(predicts_kaggel)
df_kaggel.head()

,0
0,0.228688
1,0.227828
2,0.231582
3,0.221458
4,0.203215


In [18]:
df_kaggel[0].describe()

count    19415.000000
mean         0.210803
std          0.024935
min          0.189592
25%          0.193765
50%          0.199753
75%          0.221515
max          0.482920
Name: 0, dtype: float64

In [19]:
df_kaggel = df_kaggel[0].map(lambda x: 1 if x > 0.207122 else 0);
df_kaggel = df_kaggel.to_frame()
df_kaggel["person"] = personas
df_kaggel.rename(columns = {0: 'label'},inplace = True) 

In [20]:
df_kaggel = df_kaggel[['person','label']]

### Vemos resultados y pasamos a un csv

In [21]:
df_kaggel['label'].value_counts()

0    12318
1     7097
Name: label, dtype: int64

In [22]:
df_kaggel.to_csv('Resultados.csv',index=False)

### Vemos la importancia de cada feature

In [23]:
import matplotlib.pyplot as plt
xgb.plot_importance(xg_reg)
plt.rcParams['figure.figsize'] = [30, 30]
plt.show()

<Figure size 640x480 with 1 Axes>

In [24]:
#Pruebas anteriores
#marcas + condicion + color + storage + promedio dias+ promedio horas = malo (color aportaba muy poco) #0.74"
#marcas + condicion + storage + promedio dias+ promedio horas = safa (0.76110)
#condicion + storage + promedio dias+ promedio horas =  (0.76409)
#condicion + promedio dias+ promedio horas =  (0.76688)
# 6759
#
#